# Robot Taxi - backend

In [1]:
#ros2 launch turtlebot3_gazebo turtlebot3_world.launch.py

#ros2 launch turtlebot3_navigation2 navigation2.launch.py use_sim_time:=True map:=/home/ubuntu/turtlebot3_ws/src/jupyter_notebooks/map.yaml



In [2]:


import rclpy
from rclpy.node import Node
from geometry_msgs.msg import Twist
from geometry_msgs.msg import PoseWithCovarianceStamped
from nav2_msgs.action import NavigateToPose
from rclpy.action import ActionClient
from geometry_msgs.msg import PoseStamped



import sqlite3
import time
from pathlib import Path



from storages import BaseStorage


BASE_DIR = Path("/home/ubuntu/turtlebot3_ws/src/jupyter_notebooks")

storage_obj = BaseStorage(BASE_DIR)
with storage_obj as s:
    s.create_tables_if_not_exist() 
    cursor = s.conn.cursor()
    rows = cursor.execute(f"UPDATE schedules SET is_completed=1, in_progress=0")
    s.conn.commit()

In [3]:
from types import MethodType


class Taxi(Node):
    def __init__(self): 
        super().__init__('taxi')

        self.is_robot_busy = False
        
        self.publisher_ = self.create_publisher(Twist, 
                                                'cmd_vel',
                                                10)
        

 
        self.timer = self.create_timer(
            1.0,
            self.read_db)
    
        
    def print_feedback_goal(self, future):
        try:
            print(future.feedback.x)
            print(".", end="")
            pass
        except:
            pass

    def history_success(self, future):

        print(",,-----")
        print(future.done())
        print(future.result())
        print("...-----")
        
        BASE_DIR = Path("/home/ubuntu/turtlebot3_ws/src/jupyter_notebooks")
        
        storage_obj = BaseStorage(BASE_DIR)
        with storage_obj as s:
            cursor = s.conn.cursor()
            rows = cursor.execute(f"UPDATE schedules SET is_completed=1, in_progress=0 WHERE id={self.goal_id}")
            s.conn.commit()

        self.is_robot_busy = False


    def received_task(self, future):
        BASE_DIR = Path("/home/ubuntu/turtlebot3_ws/src/jupyter_notebooks")
        
        storage_obj = BaseStorage(BASE_DIR)
        
        goal_handle = future.result()

        if not goal_handle.accepted:
            print("not accepted")
            return
        else:
            print("accepted goal")
            self.is_robot_busy = True
            with storage_obj as s:
                print(self.goal_id)
                cursor = s.conn.cursor()
                rows = cursor.execute(f"UPDATE schedules SET is_completed=0, in_progress=1 WHERE id={self.goal_id}")
                s.conn.commit()

        get_result_future = goal_handle.get_result_async()
        get_result_future.add_done_callback(self.history_success)


    def read_db(self):
        navigate_action = ActionClient(
            taxi, 
            NavigateToPose,
            '/navigate_to_pose')


        BASE_DIR = Path("/home/ubuntu/turtlebot3_ws/src/jupyter_notebooks")
        storage_obj = BaseStorage(BASE_DIR)
        with storage_obj as s:
            cursor = s.conn.cursor()
            rows = cursor.execute("SELECT * FROM schedules WHERE is_completed=0 AND in_progress=0 ORDER BY timestamp").fetchall()

            print(rows)

        if len(rows) > 0 and not self.is_robot_busy:
            navigate_pose_goal = NavigateToPose.Goal()    
            goal_pose = PoseStamped()
            goal_pose.header.stamp = self.get_clock().now().to_msg()
            goal_pose.pose.position.x = float(rows[0]["x"])
            goal_pose.pose.position.y = float(rows[0]["y"])
            goal_pose.pose.orientation.z = 0.0
            goal_pose.pose.orientation.w = 0.0
            navigate_pose_goal.pose = goal_pose

            self.goal_id = rows[0]["id"]

            goal_future_pose = navigate_action.send_goal_async(navigate_pose_goal,feedback_callback=self.print_feedback_goal)
            goal_future_pose.add_done_callback(self.received_task)

        
rclpy.init()
        
taxi = Taxi()
        
send_start_pose_publisher = taxi.create_publisher(PoseWithCovarianceStamped, 'initialpose', 1)


initial_pose = PoseWithCovarianceStamped()
initial_pose.header.stamp = taxi.get_clock().now().to_msg()
initial_pose.header.frame_id = "map"
initial_pose.pose.pose.position.x = 1.0
initial_pose.pose.pose.position.y = 3.0
initial_pose.pose.pose.orientation.z = 0.0
initial_pose.pose.pose.orientation.w = 1.0

initial_pose.pose.covariance[0] = 0.25
initial_pose.pose.covariance[7] = 0.25
initial_pose.pose.covariance[-1] = 0.06

send_start_pose_publisher.publish(initial_pose)

In [ ]:
rclpy.spin(taxi)

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
accepted goal
75
[]
,,-----
True
nav2_msgs.action.NavigateToPose_GetResult_Response(status=6, result=nav2_msgs.action.NavigateToPose_Result(result=std_msgs.msg.Empty()))
...-----
accepted goal
76
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
,,-----
True
nav2_msgs.action.NavigateToPose_GetResult_Response(status=6, result=nav2_msgs.action.NavigateToPose_Result(result=std_msgs.msg.Empty()))
...-----
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
